In [1]:
import random
import numpy as np

In [4]:
def genetic_backpack(N, max_capacity, w, c):
    n_chromo = 4
    dna = np.zeros(N*n_chromo).reshape(n_chromo, N)
    # генерируем хромосомы
    for i in range(n_chromo):
        for j in range(N):
            dna[i][j] = random.randint(0,1)
    # Считаем вес и стоимость
    total_chromo = []
    for i in range(n_chromo):
        temp_w = 0
        temp_p = 0
        for j in range(N):
            if dna[i][j]:
                temp_w += w[j]
                temp_p += c[j]
        total_chromo.append((temp_w, temp_p))
    # Сортируем
    total_chromo = sorted(total_chromo, key=lambda x: x[1], reverse=True)
    print(total_chromo)
    return

In [5]:
W = [23, 31, 29, 44, 53, 38, 63, 85, 89, 82]
C = [92, 57, 49, 68, 60, 43, 67, 84, 87, 72]
#items, cap, profit = genetic_backpack(10, 165, W, C)
genetic_backpack(10, 165, W, C)

[(273, 395), (312, 332), (265, 305), (272, 269)]


In [ ]:
def get_staticstics_backpack(algorithm):
    
    filenames = os.listdir(str(os.getcwd() + '\\benchmarks'))
    for filename in filenames:
        print(filename, '\n')
        with open(os.getcwd() + '\\benchmarks\\' + filename, 'r', encoding='utf-8') as f:
            N = int(f.readline())
            max_capacity = int(f.readline())
            w = f.readline().replace("\n", "").split(' ')
            c = f.readline().replace("\n", "").split(' ')
            for i in range(len(w)):
                w[i] = int(w[i])
            for i in range(len(c)):
                c[i] = int(c[i])
            times = []
            for i in range(10):
                items, cap, profit, t, d = algorithm(N, max_capacity, w, c)
                times.append(t)
            items, cap, profit, t, d = algorithm(N, max_capacity, w, c)
            print('Items: ', items)
            print('Final capacity: ',  cap)
            print('Profit: ', profit)
            print('Average working time: ', "{0:.3f}".format(mean(times)), 'ms')
            print('Interim solutions: ', d, '\n\n')